In [81]:
import neo4j

import csv

import math
import numpy as np
import pandas as pd

import psycopg2

In [82]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)
    

In [83]:
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","ucb_mids_w205"))

In [84]:
session = driver.session(database="neo4j")

In [85]:
def my_neo4j_wipe_out_database():
    "wipe out database by deleting all nodes and relationships"
    
    query = "match (node)-[relationship]->() delete node, relationship"
    session.run(query)
    
    query = "match (node) delete node"
    session.run(query)

In [86]:
def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

In [87]:
def my_neo4j_number_nodes_relationships():
    "print the number of nodes and relationships"
   
    
    query = """
        match (n) 
        return n.name as node_name, labels(n) as labels
        order by n.name
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    
    
    query = """
        match (n1)-[r]->(n2) 
        return n1.name as node_name_1, labels(n1) as node_1_labels, 
            type(r) as relationship_type, n2.name as node_name_2, labels(n2) as node_2_labels
        order by node_name_1, node_name_2
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_relationships = df.shape[0]
    
    print("-------------------------")
    print("  Nodes:", number_nodes)
    print("  Relationships:", number_relationships)
    print("-------------------------")


In [88]:
def my_neo4j_create_node(station_name):
    "create a node with label Station"
    
    query = """
    
    CREATE (:Station {name: $station_name})
    
    """
    
    session.run(query, station_name=station_name)
    

In [89]:
def my_neo4j_create_relationship_one_way(from_station, to_station, weight):
    "create a relationship one way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)
    

In [90]:
def my_neo4j_create_relationship_two_way(from_station, to_station, weight):
    "create relationships two way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to),
           (to)-[:LINK {weight: $weight}]->(from)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)
    

In [91]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [92]:
cursor = connection.cursor()

# Wipe out Neo4j data base

In [93]:
my_neo4j_wipe_out_database()

In [94]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 0
  Relationships: 0
-------------------------


# Creating Graph

In [95]:
connection.rollback()

query = """

select station
from stations
order by station

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    
    my_neo4j_create_node('depart ' + station)
    my_neo4j_create_node('arrive ' + station)
    

In [96]:
connection.rollback()

query = """

select station, line
from lines
order by station, line

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    line = row[1]
    
    depart = 'depart ' + station
    arrive = 'arrive ' + station
    line_station = line + ' ' + station
    
    my_neo4j_create_node(line_station)
    my_neo4j_create_relationship_one_way(depart, line_station, 0)
    my_neo4j_create_relationship_one_way(line_station, arrive, 0)
    

In [97]:
connection.rollback()

query = """

select a.station, a.line as from_line, b.line as to_line, s.transfer_time
from lines a
     join lines b
       on a.station = b.station and a.line <> b.line 
     join stations s
       on a.station = s.station
order by 1, 2, 3

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    from_line = row[1]
    to_line = row[2]
    transfer_time = int(row[3])
    
    from_station = from_line + ' ' + station
    to_station = to_line + ' ' + station
    
    my_neo4j_create_relationship_one_way(from_station, to_station, transfer_time)

In [98]:
connection.rollback()

query = """

select a.line, a.station as from_station, b.station as to_station, t.travel_time
from lines a
  join lines b
    on a.line = b.line and b.sequence = (a.sequence + 1)
  join travel_times t
    on (a.station = t.station_1 and b.station = t.station_2)
        or (a.station = t.station_2 and b.station = t.station_1)
order by line, from_station, to_station

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    line = row[0]
    from_station = line + ' ' + row[1]
    to_station = line + ' ' + row[2]
    travel_time = int(row[3])
    
    my_neo4j_create_relationship_two_way(from_station, to_station, travel_time)
    

The output should look similar to this:

```
-------------------------
  Nodes: 214
  Relationships: 652
-------------------------
```

In [119]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 214
  Relationships: 652
-------------------------


# Yen's Algorithm to find K Shortest Paths from Point A to B

In [156]:
def my_neo4j_shortest_path_yens(from_station, to_station, k):
    """
    Given a from station and to station, run and print the k-shortest paths using Yen's algorithm.
    """
   
    session.run("CALL gds.graph.drop('ds_graph', false)")
    session.run("CALL gds.graph.project('ds_graph', 'Station', 'LINK', {relationshipProperties: 'weight'})")

    query = """
    MATCH (source:Station {name: $source}), (target:Station {name: $target})
    CALL gds.shortestPath.yens.stream('ds_graph', {
        sourceNode: source,
        targetNode: target,
        k: $k,
        relationshipWeightProperty: 'weight'
    })
    YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
    RETURN
        index,
        gds.util.asNode(sourceNode).name AS from,
        gds.util.asNode(targetNode).name AS to,
        totalCost,
        [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodes,
        costs
    ORDER BY index
    """

    result = session.run(query, source=from_station, target=to_station, k=k)

    # Printing the Results
    for r in result:
        total_cost = int(r['totalCost'])
        print("\n--------------------------------")
        print(f"Path {r['index'] + 1}:")
        print("From: ", r['from'])
        print("To: ", r['to'])
        print("Total Cost: ", total_cost)
        print("Minutes: ", round(total_cost / 60.0, 1))
        print("Path: ", " -> ".join(r['nodes']))
        print("--------------------------------")

## Finding the shortest paths from SFO to OAK

In [157]:
my_neo4j_shortest_path_yens('depart SFO', 'arrive OAK', 4)


--------------------------------
Path 1:
From:  depart SFO
To:  arrive OAK
Total Cost:  3882
Minutes:  64.7
Path:  depart SFO -> yellow SFO -> yellow San Bruno -> yellow South San Francisco -> yellow Colma -> yellow Daly City -> yellow Balboa Park -> green Balboa Park -> green Glen Park -> green 24th Street Mission -> green 16th Street Mission -> green Civic Center -> green Powell Street -> green Montgomery Street -> green Embarcadero -> green West Oakland -> green Lake Merritt -> green Fruitvale -> green Coliseum -> gray Coliseum -> gray OAK -> arrive OAK
--------------------------------

--------------------------------
Path 2:
From:  depart SFO
To:  arrive OAK
Total Cost:  3882
Minutes:  64.7
Path:  depart SFO -> yellow SFO -> yellow San Bruno -> yellow South San Francisco -> yellow Colma -> yellow Daly City -> yellow Balboa Park -> blue Balboa Park -> blue Glen Park -> blue 24th Street Mission -> blue 16th Street Mission -> blue Civic Center -> blue Powell Street -> blue Montgomer

# Betweenness Centrality

In [158]:
def betweeness():
    query = "CALL gds.graph.drop('ds_graph', false)"
    session.run(query)
    query = "CALL gds.graph.project('ds_graph', 'Station', 'LINK', {relationshipProperties: 'weight'})"
    session.run(query)
    
    query = """
    CALL gds.betweenness.stream('ds_graph', {relationshipWeightProperty: 'weight'})
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).name AS name, score as betweenness
    ORDER BY betweenness DESC

    """

    return my_neo4j_run_query_pandas(query)
    

In [159]:
betweeness()

,name,betweenness
0,yellow MacArthur,6531.833333
1,orange Coliseum,5765.666667
2,orange MacArthur,5585.833333
3,yellow Rockridge,5509.000000
4,orange Lake Merritt,4998.000000
...,...,...
209,arrive Warm Springs,0.000000
210,depart West Dublin,0.000000
211,arrive West Dublin,0.000000
212,depart West Oakland,0.000000


# Louivain Communities 

In [165]:
def louivain_communities():
    query = "CALL gds.graph.drop('ds_graph', false)"
    session.run(query)
    query = "CALL gds.graph.project('ds_graph', 'Station', 'LINK', {relationshipProperties: 'weight'})"
    session.run(query)
    
    query = """

    CALL gds.louvain.stream('ds_graph', {includeIntermediateCommunities: true})
    YIELD nodeId, communityId, intermediateCommunityIds
    RETURN gds.util.asNode(nodeId).name AS name, communityId as community, intermediateCommunityIds as intermediate_community
    ORDER BY community, name ASC

    """

    return my_neo4j_run_query_pandas(query)  

In [166]:
louivain_communities()

,name,community,intermediate_community
0,arrive 16th Street Mission,7,"[3, 7, 7]"
1,arrive 24th Street Mission,7,"[7, 7, 7]"
2,blue 16th Street Mission,7,"[3, 7, 7]"
3,blue 24th Street Mission,7,"[7, 7, 7]"
4,depart 16th Street Mission,7,"[3, 7, 7]"
...,...,...,...
209,orange Berryessa,91,"[17, 57, 91]"
210,orange Fremont,91,"[41, 91, 91]"
211,orange Milpitas,91,"[57, 57, 91]"
212,orange Union City,91,"[91, 91, 91]"
